cannon 3
---

In [1]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
%matplotlib inline
import tensorflow as tf
import keras

Simulation function

In [2]:
pi = np.pi

#simulates a ballistic trajectory and returns the smallest distance to the target position
def simulate_trajectory_distance(speed, azi, pitch, target, origin = np.array([0.,0.,0.]), timestep = 0.1, g = -10.):
    #get cartesian velocity from spherical coordinates
    velocity = speed*np.array([
        np.cos(azi)*np.cos(pitch),
        np.sin(azi)*np.cos(pitch),
        np.sin(pitch)
    ])
#     print(f'Starting velocity {velocity}.')
    #make acceleration vector
    acc = np.array([0.,0., g])
    #record origin and first position after 1 timestep
    history = [origin, origin+ velocity*timestep + (1/2)*acc*timestep**2]
    position = history[-1]
#     print(history)
    #record distances
    distances = [np.linalg.norm(target - origin), np.linalg.norm(target - position)]
    #while the ball is above the ground, timestep to change position
    while position[2] >= 0:
#         print(history[-1])
        position = 2*history[-1] - history[-2] + acc*(timestep**2)
        distances.append(np.linalg.norm(target - position))
        history.append(position)
    return np.array(distances).min()
    
def tf_simulate_trajectory(speed, azi, pitch, target, origin = tf.constant([0.,0.,0.], dtype = 'float32'), timestep = 0.1, g = -10., distance_only = True):
    #get cartesian velocity from spherical coordinates
    velocity = speed*tf.convert_to_tensor([
        tf.math.cos(azi)*tf.math.cos(pitch),
        tf.math.sin(azi)*tf.math.cos(pitch),
        tf.math.sin(pitch)
    ])
    acc = tf.constant([0.,0., g], dtype='float32')
    history = [origin, origin+ velocity*timestep + (1/2)*acc*timestep**2]
    position = history[-1]
    distances = [
        tf.norm(target - origin),
        tf.norm(target - position)
    ]
    #while the ball is above the ground, timestep to change position
    while position[2] >= 0:
#         print(history[-1])
        position = 2*history[-1] - history[-2] + acc*(timestep**2)
        distances.append(tf.norm(target - position))
        history.append(position)
    if distance_only:
        return tf.reduce_min(tf.convert_to_tensor(distances))
    else:
        return tf.convert_to_tensor(history)    

In [3]:
distance = tf_simulate_trajectory(100., 0., pi/3, tf.constant([100.,200.,300.], dtype='float32'))
distance

<tf.Tensor: shape=(), dtype=float32, numpy=221.16388>

In [4]:
loss_fn = tf.losses.Huber()
loss_fn([0.],[distance])

<tf.Tensor: shape=(), dtype=float32, numpy=220.66388>

In [5]:
seed = 2022
rng = np.random.default_rng(seed)

#eps == 1.1920928955078125e-07
eps = np.finfo(np.float32).eps.item()

gamma = 0.99
rounds_per_load = 3
maximum_acceptable_loss = 500.
scores_to_average = 3
max_speed = 100. #m/s
field_size = 2000. #m

hit_distance = 5 #5m radius of target
#miss_penalty = 500 #add this distance as a penalty for missing
num_inputs = 3 #target x, y, z
num_outputs = 3 #speed/max_speed, azimuth/(2pi), pitch/(pi)(
num_hidden = 32

#make model
# inputs = keras.layers.Input(shape = (num_inputs,))
# dense_1 = keras.layers.Dense(num_hidden, activation = 'relu', kernel_initializer=tf.initializers.he_normal)(inputs)
# outputs = keras.layers.Dense(num_outputs)(dense_1)
# model = keras.Model(inputs = inputs, outputs = outputs)

inputs = keras.layers.Input(shape = (num_inputs,))
dense_1 = keras.layers.Dense(num_hidden, activation = 'relu', bias_initializer=tf.initializers.he_normal, kernel_initializer=tf.initializers.he_normal)(inputs)
outputs = keras.layers.Dense(num_outputs)(dense_1)
model = keras.Model(inputs = inputs, outputs = outputs)

#optimizer and loss function
optimizer = tf.optimizers.Adam(learning_rate = 0.05)
loss_fn = tf.losses.Huber()

avg_recent_loss = 1000000.
all_scores = []
total_loss = tf.constant(0.)

# model.trainable_variables[1] = tf.convert_to

while avg_recent_loss > maximum_acceptable_loss:
    with tf.GradientTape() as tape:
        command_history = []
        loss_history = []
        #reload the cannon and shoot all shots
        for shot in range(rounds_per_load):
#             print(f'round {shot}')
            target = field_size*rng.random(3) - field_size/2
            target[2] = abs(target[2])
            target = tf.convert_to_tensor(target, dtype = 'float32')
            #this just makes it so the model doesn't think these are separate collections of inputs
            target = tf.expand_dims(target, 0)
            
            command = model(target)
            speed, azi, pitch = command[0]
            speed = speed*max_speed
            azi = azi*2*pi
            pitch = pitch*pi
            
            loss = tf_simulate_trajectory(speed, azi, pitch, target, timestep = 2)
            loss = loss_fn([0.], [loss])
            loss_history.append(loss)
            
        total_loss = sum(loss_history)
        grads = tape.gradient(total_loss, model.trainable_variables)
        if grads == [None, None, None, None]:
            grads = [-.1,.1,-.2,.2]

    print('got to optimizing')
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    all_scores.append(total_loss)
    loss_history.clear()
    avg_recent_loss = tf.reduce_mean(all_scores[-scores_to_average:])
    if len(all_scores)%1 == 0:
        print(f'finished with trial {len(all_scores)}')
        print(f'avg loss: {avg_recent_loss}')

got to optimizing
finished with trial 1
avg loss: 3260.6982421875
got to optimizing
finished with trial 2
avg loss: 3301.713623046875
got to optimizing
finished with trial 3
avg loss: 3451.8203125


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\David\AppData\Local\Temp/ipykernel_5924/3265140256.py", line 60, in <module>
    loss = tf_simulate_trajectory(speed, azi, pitch, target, timestep = 2)
  File "C:\Users\David\AppData\Local\Temp/ipykernel_5924/2838888265.py", line 43, in tf_simulate_trajectory
    while position[2] >= 0:
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1134, in __bool__
    return bool(self._numpy())
  File "C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\ops.py", line 1189, in _numpy
    return self._numpy_internal()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py", lin

TypeError: object of type 'NoneType' has no len()

In [ ]:
all_scores

In [ ]:
grads